In [2]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='data/Planetoid', name='Cora',
					transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}')
print('=======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')
data = dataset[0]

print()
print(data)
print('=======================')
print(f'Number of Nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree :{data.num_edges / data.num_nodes:2f}')



Dataset: Cora()
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of Nodes: 2708
Number of edges: 10556
Average node degree :3.898080


In [3]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def visualize(h, color):
	z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())
	plt.figure(figsize=(10,10))
	plt.xticks([])
	plt.yticks([])
	plt.scatter(z[:,0], z[:,1], s=70, c=color, cmap='Set2')
	plt.show()

#### 传统神经网络求解

In [4]:
import torch
from torch.nn import Linear
import torch.nn.functional as F

class MLP(torch.nn.Module):
	def __init__(self, hidden_channels):
		super().__init__()
		torch.manual_seed(12345)
		# 指定 2 个全连接层
		self.lin1 = Linear(dataset.num_features, hidden_channels)
		self.lin2 = Linear(hidden_channels, dataset.num_classes)

	def forward(self, x):
		x = self.lin1(x)
		x = x.relu()
		x = F.dropout(x, p=0.5, training=self.training)
		x = self.lin2(x)
		return x

model = MLP(hidden_channels=16)
print(model)

MLP(
  (lin1): Linear(in_features=1433, out_features=16, bias=True)
  (lin2): Linear(in_features=16, out_features=7, bias=True)
)


In [5]:
model = MLP(hidden_channels=16)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, 
							 weight_decay=5e-4)

def train():
	model.train()
	optimizer.zero_grad() # 梯度归零
	out = model(data.x)
	loss = criterion(out[data.train_mask], data.y[data.train_mask])
	loss.backward()
	optimizer.step()
	return loss

def test():
	model.eval()
	out = model(data.x)
	test_correct = criterion(out[data.train_mask] == data.y[data.train_mask])
	test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
	return test_acc

for epoch in range(1, 201):
	loss = train()
	print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 1.9615
Epoch: 002, Loss: 1.9557
Epoch: 003, Loss: 1.9505
Epoch: 004, Loss: 1.9423
Epoch: 005, Loss: 1.9327
Epoch: 006, Loss: 1.9279
Epoch: 007, Loss: 1.9144
Epoch: 008, Loss: 1.9087
Epoch: 009, Loss: 1.9023
Epoch: 010, Loss: 1.8893
Epoch: 011, Loss: 1.8776
Epoch: 012, Loss: 1.8594
Epoch: 013, Loss: 1.8457
Epoch: 014, Loss: 1.8365
Epoch: 015, Loss: 1.8280
Epoch: 016, Loss: 1.7965
Epoch: 017, Loss: 1.7984
Epoch: 018, Loss: 1.7832
Epoch: 019, Loss: 1.7495
Epoch: 020, Loss: 1.7441
Epoch: 021, Loss: 1.7188
Epoch: 022, Loss: 1.7124
Epoch: 023, Loss: 1.6785
Epoch: 024, Loss: 1.6660
Epoch: 025, Loss: 1.6119
Epoch: 026, Loss: 1.6236
Epoch: 027, Loss: 1.5827
Epoch: 028, Loss: 1.5784
Epoch: 029, Loss: 1.5524
Epoch: 030, Loss: 1.5020
Epoch: 031, Loss: 1.5065
Epoch: 032, Loss: 1.4742
Epoch: 033, Loss: 1.4581
Epoch: 034, Loss: 1.4246
Epoch: 035, Loss: 1.4131
Epoch: 036, Loss: 1.4112
Epoch: 037, Loss: 1.3923
Epoch: 038, Loss: 1.3055
Epoch: 039, Loss: 1.2982
Epoch: 040, Loss: 1.2543
